In [1]:
#from helper import differential_equation_loss, optimizer_function_factory
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
# el data set
import tensorflow as tf
# Ruta del archivo TFRecord que deseas cargar
input_path = "dataset.tfrecord"


# Función para parsear un ejemplo TFRecord
def parse_dataset(example_proto):
    feature_description = {
        'x': tf.io.FixedLenFeature([3], tf.float32),
        'v': tf.io.FixedLenFeature([3], tf.float32)
    }
    dataset = tf.io.parse_single_example(example_proto, feature_description)
    return dataset['x'], dataset['v']

# Cargar el dataset desde el archivo TFRecord
dataset = tf.data.TFRecordDataset(input_path)
dataset = dataset.map(parse_dataset)


def head_dataset(dataset):
    # Ahora puedes iterar sobre el dataset para leer los datos
    print_n = 5
    for data in dataset:
        print(data)
        print_n -= 1
        if print_n < 0:
            break

head_dataset(dataset)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.9167395, 3.805452 , 3.721134 ], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.3183789 , -0.934036  , -0.75950456], dtype=float32)>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.805452 , 3.721134 , 3.6490405], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.934036  , -0.75950456, -0.67820454], dtype=float32)>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.721134 , 3.6490405, 3.582606 ], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.75950456, -0.67820454, -0.64235467], dtype=float32)>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.6490405, 3.582606 , 3.5185351], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.67820454, -0.64235467, -0.6291701 ], dtype=float32)>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3.582606 , 3.5185351, 3.4551263], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy

In [3]:
# Define el tamaño del lote (batch)
batch_size = 32

# Mezcla los datos y crea lotes
dataset = dataset.shuffle(buffer_size=1000)  # Opcional: si deseas mezclar los datos
dataset = dataset.batch(batch_size)

In [4]:
def lendataset(dataset):
    size = 0
    for batch_x, batch_v in dataset:
        # Realiza operaciones con el lote de datos
        size += batch_x.shape[0]
    return size
    
lendataset(dataset)

49000

## El modelo
El modelo tiene una sola entrada y una sola salida; quiere decir que es necesario conocer $x_k$ para entonces predecir $x_{k+1}$, el modelo se encarga de predecir el siguiente paso temporal,
Este hecho permite calcular el momentum a partir del modelo; por tanto hay varios constrains a tener en cuenta para optimizar esto:

- $$p = \frac{dx}{dt} = \frac{x_{k+1} - x_{k-1}}{2h}$$

- $$\frac{d^2x}{dt^2} = \frac{x_{k+1} - 2 x_k + x_{k-1}}{h^2}$$

## bloques residuales

In [5]:
head_dataset(dataset)

(<tf.Tensor: shape=(32, 3), dtype=float32, numpy=
array([[-1.5815749 , -1.4789399 , -1.3675904 ],
       [ 3.605717  ,  3.551124  ,  3.4933321 ],
       [-1.9911541 , -2.0188694 , -2.0231972 ],
       [-0.07017034, -0.287925  , -0.5126584 ],
       [ 1.3852948 ,  1.5765079 ,  1.7382947 ],
       [-0.6656486 , -0.4902728 , -0.30309474],
       [ 3.2619092 ,  3.1954124 ,  3.1275594 ],
       [ 2.0550628 ,  2.0733063 ,  2.0692742 ],
       [-1.1251506 , -0.98242646, -0.8294743 ],
       [ 0.39295   ,  0.63163394,  0.8708254 ],
       [-2.7657938 , -2.6791174 , -2.5921326 ],
       [ 1.9211241 ,  1.8104981 ,  1.694929  ],
       [ 0.5996897 ,  0.82701164,  1.0495907 ],
       [ 1.6927599 ,  1.8317155 ,  1.937518  ],
       [-0.51919776, -0.33750966, -0.14394082],
       [ 0.22991054,  0.4527352 ,  0.6792423 ],
       [-0.30309474, -0.10384133,  0.10683315],
       [ 1.1907692 ,  1.0564871 ,  0.91208214],
       [-1.7939249 , -1.7297784 , -1.6548002 ],
       [ 1.432203  ,  1.3136535 ,  1.1

In [50]:
# Supongamos que tienes 1000 elementos en tu dataset
total_size = lendataset(dataset)
# Supongamos que tienes un tamaño de lote (batch_size) definido previamente
batch_size = 32

# Define el tamaño de los conjuntos de entrenamiento, validación y prueba
train_p = 0.8
valid_p = 0.1
test_p = 1 - train_p - valid_p

train_size = int( total_size * train_p )
valid_size = int( total_size * valid_p )
test_size = int( total_size * test_p )

# Divide el dataset en tres conjuntos
train_dataset = dataset.take(train_size // batch_size)
valid_dataset = dataset.skip(train_size // batch_size).take(valid_size // batch_size)

test_dataset = dataset.skip(
    train_size // batch_size + valid_size // batch_size).take(test_size // batch_size)


print("train size", lendataset(train_dataset))
print("valid size", lendataset(valid_dataset))
print("test size", lendataset(test_dataset))


train size 39200
valid size 4896
test size 4896


In [61]:
train_dataset

(<tf.Tensor: shape=(1225, 32, 3), dtype=float32, numpy=
 array([[[-1.18310821e+00, -1.38308847e+00, -1.56088507e+00],
         [ 2.80720979e-01,  8.40639547e-02, -1.24016054e-01],
         [-7.46678634e-05,  2.27418467e-01,  4.62978810e-01],
         ...,
         [-1.25822902e+00, -1.12515056e+00, -9.82426465e-01],
         [-1.59050012e+00, -1.48945868e+00, -1.37970924e+00],
         [-2.07178426e+00, -2.07377982e+00, -2.05593204e+00]],
 
        [[ 2.01470375e+00,  1.99136055e+00,  1.95231640e+00],
         [ 1.43132639e+00,  1.31551242e+00,  1.19076920e+00],
         [ 1.05018365e+00,  1.27217758e+00,  1.47426498e+00],
         ...,
         [-4.37876105e-01, -6.65072203e-01, -8.92044902e-01],
         [ 1.08455312e+00,  9.43797827e-01,  7.92372465e-01],
         [ 2.81536674e+00,  2.84546256e+00,  2.84199762e+00]],
 
        [[-4.04996157e-01, -2.15171486e-01, -1.39176268e-02],
         [ 1.82094347e+00,  1.73675585e+00,  1.64369583e+00],
         [-1.85426235e+00, -1.93711412e+00

In [51]:
head_dataset(train_dataset)

(<tf.Tensor: shape=(32, 3), dtype=float32, numpy=
array([[ 0.09231405, -0.11789805, -0.33724973],
       [-0.16521749,  0.03809637,  0.2512436 ],
       [ 0.63163394,  0.8708254 ,  1.1041514 ],
       [-1.8328947 , -1.9225457 , -1.9828311 ],
       [-0.12401605, -0.34122723, -0.56421053],
       [-1.317452  , -1.502453  , -1.662602  ],
       [ 1.7956738 ,  1.70867   ,  1.6127248 ],
       [ 3.038803  ,  2.9674733 ,  2.894313  ],
       [-2.338824  , -2.2487338 , -2.1545565 ],
       [ 2.0752487 ,  2.0577796 ,  2.023416  ],
       [-1.1565461 , -1.0012561 , -0.8353659 ],
       [ 0.22387747,  0.02271271, -0.18908694],
       [ 0.8840941 ,  1.1181078 ,  1.3379033 ],
       [-1.3367721 , -1.2139884 , -1.0818899 ],
       [-0.6650722 , -0.8920449 , -1.1119938 ],
       [-1.4066192 , -1.2896832 , -1.1635668 ],
       [-2.1952088 , -2.0966284 , -1.993914  ],
       [-1.1146762 , -0.9574621 , -0.78970873],
       [ 0.8822524 ,  0.7253202 ,  0.5571646 ],
       [ 1.516491  ,  1.4083143 ,  1.2

In [52]:
# Convierte las listas de tuplas en tensores
def list_of_tuples_to_tensors(dataset_list):
    x_list, v_list = zip(*dataset_list)
    x_tensor = tf.convert_to_tensor(x_list, dtype=tf.float32)
    v_tensor = tf.convert_to_tensor(v_list, dtype=tf.float32)
    return x_tensor, v_tensor

train_dataset = list_of_tuples_to_tensors(train_dataset)
valid_dataset = list_of_tuples_to_tensors(valid_dataset)
test_dataset = list_of_tuples_to_tensors(test_dataset)

In [64]:
xk_train = train_dataset[0][:, :, 0]
xkp1_train = train_dataset[0][:, :,1]
xkp2_train = train_dataset[0][:, :,2]
dxdt_train = train_dataset[1][:, :,1]

In [70]:
xk_valid = valid_dataset[0][:, :, 0]
xkp1_valid = valid_dataset[0][:, :,1]
xkp2_valid = valid_dataset[0][:, :,2]
dxdt_valid = valid_dataset[1][:, :,1]

In [ ]:
xk_test = test_dataset[0][:, :, 0]
xkp1_test = test_dataset[0][:, :,1]
xkp2_test = test_dataset[0][:, :,2]
dxdt_test = test_dataset[1][:, :,1]

# Model

In [66]:
class ResidualBlock(layers.Layer):
    def __init__(self, n_layers=4, n_neuron=64, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.inputlayer = layers.Dense(1, activation=None)
        self.hidden = [
            layers.Dense(n_neuron, activation=activation,
                kernel_initializer="he_normal")
            for _ in range(n_layers)
        ]
        self.outputlayer = layers.Dense(1, activation=None)
    
    def call(self, inputs):
        Z = self.inputlayer(inputs)
        for layer in self.hidden:
            Z = layer(Z)
        return self.outputlayer(Z) + inputs

In [67]:
resblock =  ResidualBlock(4, 64, 'tanh')
resblock(xk_train)

<tf.Tensor: shape=(1225, 32), dtype=float32, numpy=
array([[-0.90901744,  0.5548117 ,  0.27401608, ..., -0.98413825,
        -1.3164093 , -1.7976935 ],
       [ 2.3147473 ,  1.73137   ,  1.3502271 , ..., -0.13783258,
         1.3845966 ,  3.1154103 ],
       [-0.1311562 ,  2.0947833 , -1.5804224 , ...,  2.12646   ,
        -1.7081649 , -0.7886275 ],
       ...,
       [ 0.8362661 ,  3.2297907 , -1.3224454 , ...,  1.7862229 ,
         1.7332697 ,  2.170337  ],
       [-0.36863798, -1.5162919 ,  0.6973914 , ...,  1.9156845 ,
        -3.850663  , -1.4806514 ],
       [-2.2804637 , -0.51451325, -0.05708876, ..., -3.0569625 ,
         1.5590454 , -1.5446255 ]], dtype=float32)>

In [68]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.norm_layer = layers.Normalization()
        #self.flatten = layers.Flatten()  # Aplana los datos
        self.input_layer = layers.Dense(1, activation=None)
        self.resblock = ResidualBlock(4, 64, 'tanh')

    def call(self, inputs):
        print(inputs.shape)
        # Extrae el primer valor de "x" para cada dato en la forma (batch_size, 2, 800)
        xk = self.input_layer(inputs)
        print(xk.shape)
        #xk = inputs  # Opción 1: primer valor
        #xk = inputs
        # calcula x_k+1
        xkp1 = self.resblock(xk) #+ xk
        print(xkp1.shape)
        # calcula x_k+2
        xkp2 = self.resblock(xkp1) #+ xkp1
        
        # derivada centrada
        # f' = (f(x+h) - f(x-h) )/ 2h + O(h^2)
        # asumimos h=1
        dxdt = ( xkp2 - xk )/2

        return [xkp1, xkp2, dxdt]


model = ResidualRegressor()
# ... model build

In [71]:
# Compila el modelo (puedes ajustar el optimizador y la función de pérdida según tus necesidades)
model.compile(optimizer='adam', loss='mse')

# Entrena el modelo
model.fit(xk_train, [xkp1_train, xkp2_train, dxdt_train], epochs=10, validation_split=0.1)



Epoch 1/10
(None, 32)
(None, 1)
(None, 1)
(None, 32)
(None, 1)
(None, 1)
25/35 [====================>.........] - ETA: 0s - loss: 9.0472 - output_1_loss: 3.3866 - output_2_loss: 3.2875 - output_3_loss: 2.3731 (None, 32)
(None, 1)
(None, 1)
35/35 [==============================] - 3s 16ms/step - loss: 9.0540 - output_1_loss: 3.4012 - output_2_loss: 3.3042 - output_3_loss: 2.3487 - val_loss: 8.1224 - val_output_1_loss: 2.8644 - val_output_2_loss: 2.8254 - val_output_3_loss: 2.4327
Epoch 2/10
35/35 [==============================] - 0s 6ms/step - loss: 8.9188 - output_1_loss: 3.3776 - output_2_loss: 3.2888 - output_3_loss: 2.2524 - val_loss: 8.0298 - val_output_1_loss: 2.8754 - val_output_2_loss: 2.8156 - val_output_3_loss: 2.3388
Epoch 3/10
35/35 [==============================] - 0s 6ms/step - loss: 8.8277 - output_1_loss: 3.3705 - output_2_loss: 3.2844 - output_3_loss: 2.1728 - val_loss: 8.0132 - val_output_1_loss: 2.8627 - val_output_2_loss: 2.8284 - val_output_3_loss: 2.3221
Epoch 4/

In [72]:

# Evalúa el modelo en el conjunto de validación
validation_loss = model.evaluate(xk_valid, (xkp1_valid, xkp2_valid, dxdt_valid))
print("Validation Loss:", validation_loss)


5/5 [==============================] - 0s 5ms/step - loss: 7.8047 - output_1_loss: 2.9263 - output_2_loss: 2.8727 - output_3_loss: 2.0057
Validation Loss: [7.804656982421875, 2.9263100624084473, 2.872685432434082, 2.005661725997925]
